# Machine learning con python

# integrantes:
## Angela Rodriguez Maldonado A01636960
## Angel Marquez Hernandez A01351408

In [27]:
#codigo para empezar a trabajar, es necesario correr
import pandas as pd
df_review =pd.read_csv('IMDB Dataset.csv')
df_review

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


## desbalanceo de datos

In [28]:
# count origina de datos balanceados
df_review.value_counts('sentiment')

sentiment
negative    25000
positive    25000
dtype: int64

In [29]:
#desbalanceo de datos para ejercicio
df_positivo = df_review[df_review['sentiment'] == 'positive'][:9000]
df_negativo = df_review[df_review['sentiment'] == 'negative'][:1000]

df_review_des = pd.concat([df_positivo,df_negativo])
df_review_des.value_counts('sentiment')

sentiment
positive    9000
negative    1000
dtype: int64

In [30]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler()

#metemos la informacion en nuestro RUS, indicamos el input(reviews) y output(sentimiento) 
# el input debe ser un array 2D, por eso usamos doble llave
df_review_bal, df_review_bal['sentiment'] = rus.fit_resample(df_review_des[['review']], df_review_des['sentiment'])
df_review_bal

#nos devuelve un set balanceado de datos, en este caso 1000 positivos y 1000 negativos

,review,sentiment
0,Basically there's a family where a little boy ...,negative
1,"This show was an amazing, fresh & innovative i...",negative
2,Encouraged by the positive comments about this...,negative
3,Phil the Alien is one of those quirky films wh...,negative
4,I saw this movie when I was about 12 when it c...,negative
...,...,...
1995,Another Channel 4 great canned long before it'...,positive
1996,"Overall, this is entertaining and odd film. Do...",positive
1997,...I cannot believe I was hooked on this show ...,positive
1998,Francis Ford Coppola wrote and directed this s...,positive


In [31]:
from sklearn.model_selection import train_test_split

#metemos los datos, el porcentaje de datos para el testing y la semilla de random para poder replicar los resultados
train,test=train_test_split(df_review_bal, test_size=0.33,random_state=42)

In [32]:
train_x, train_y=train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

# bag of words
minuto 19:30 en el video

In [33]:
#Count vectorizer
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
text = ["Amo escribir codigo en Python. Amo el código en Python",
        "Odio escribir codigo en Java. Odio el código en Java"]

df = pd.DataFrame({'review': ['review1', 'review2'], 'text':text})
cv = CountVectorizer()
cv_matrix = cv.fit_transform(df['text'])
df_dtm = pd.DataFrame(cv_matrix.toarray(), index=df['review'].values, columns=cv.get_feature_names())
df_dtm

#hace un conteo de las palabras pero tiene como desventajas que no toma en cuenta la importancia de cada palabra

/Users/angelmarquez/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,amo,codigo,código,el,en,escribir,java,odio,python
review1,2,1,1,1,2,1,0,0,2
review2,0,1,1,1,2,1,2,2,0


In [34]:
#Tfidf (term frequency - inverse document frequency)
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
text = ["Amo escribir codigo en Python. Amo el código en Python",
        "Odio escribir codigo en Java. Odio el código en Java"]

df = pd.DataFrame({'review': ['review1', 'review2'], 'text':text})
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['text'])
df_dtm = pd.DataFrame(tfidf_matrix.toarray(), index=df['review'].values, columns=tfidf.get_feature_names())
df_dtm
#hace una matriz con palabras tomando en cuenta su importancia y frecuencia

/Users/angelmarquez/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,amo,codigo,código,el,en,escribir,java,odio,python
review1,0.576152,0.204969,0.204969,0.204969,0.409937,0.204969,0.000000,0.000000,0.576152
review2,0.000000,0.204969,0.204969,0.204969,0.409937,0.204969,0.576152,0.576152,0.000000


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(train_x)

test_x_vector = tfidf.transform(test_x)
#fit: encontrar parametros ideales
#transform: aplica los parametros previamente encontrados con fit

In [36]:
train_x_vector

<1340x19970 sparse matrix of type '<class 'numpy.float64'>'
	with 114698 stored elements in Compressed Sparse Row format>

# Support Vector Machines (SVM)

In [52]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(train_x_vector, train_y)



SVC(kernel='linear')

# testeo


In [38]:
#para el testeo se usa data numerica, entonces usamos tfidf para convertirlo en valores numericos
print(svc.predict(tfidf.transform(['A good movie']))) 
print(svc.predict(tfidf.transform(['An excellent movie'])))
print(svc.predict(tfidf.transform(['"I did not like this movie at all I gave this movie away"'])))

['positive']
['positive']
['negative']


# decision tree


In [39]:
# entrenamos el modelo de arbol de decisiones con la informacion que obtuvimos
from sklearn.tree import DecisionTreeClassifier

dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y)

DecisionTreeClassifier()

# Naive bayes

In [40]:
# entrenamos el modelo de Bayes con la misma informacion
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(train_x_vector.toarray(), train_y)

GaussianNB()

# Logistic Regression

In [41]:
# entrenamos el modelo de regresion logistica con la misma informacion
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_x_vector, train_y)

LogisticRegression()

# Score (accuracy)

In [42]:
#con score se ponen dos parametros. el primero es el input y el segundo el output, aqui ya se usa la data de testeo
print(svc.score(test_x_vector, test_y))
print(dec_tree.score(test_x_vector, test_y))
print(gnb.score(test_x_vector.toarray(), test_y))
print(lr.score(test_x_vector, test_y))

0.85
0.6878787878787879
0.646969696969697
0.8333333333333334


# F1 score

In [43]:
#F1 Score = 2(Recall Precision) / (Recall + Precision)
from sklearn.metrics import f1_score

f1_score(test_y, svc.predict(test_x_vector),
         labels=['positive', 'negative'],
         average=None)

#aqui sacamos calificacion en relacion a los resultados positivos y los negativos

array([0.85333333, 0.84651163])

# reporte de clasificacion

In [44]:
# eeste es como un resumen de toda la informacion que calculamos antes
from sklearn.metrics import classification_report

print(classification_report(test_y, svc.predict(test_x_vector),
                      labels=['positive', 'negative']))

              precision    recall  f1-score   support

    positive       0.85      0.86      0.85       335
    negative       0.85      0.84      0.85       325

    accuracy                           0.85       660
   macro avg       0.85      0.85      0.85       660
weighted avg       0.85      0.85      0.85       660



# matriz de confusion

In [45]:
# TP, FP
# FN, TN
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, svc.predict(test_x_vector),
                 labels=['positive', 'negative'])

array([[288,  47],
       [ 52, 273]])

# GridsearchCV

In [46]:
from sklearn.model_selection import GridSearchCV

#se hace un diccionario con dos elementos, funcionan como parametros para identificar cual es mas ideal , C es de error permitido
#como output nos da los mejores parametros
paremetros = {'C':[1,4,8,16,32], 'kernel':['linear', 'rbf']}
svc = SVC()
svc_grid = GridSearchCV(svc, paremetros, cv=5)#cv, validacion cruzada para ver cuantas validaciones se hacen
svc_grid.fit(train_x_vector, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 4, 8, 16, 32], 'kernel': ['linear', 'rbf']})

In [47]:
print(svc_grid.best_estimator_)
print(svc_grid.best_params_)

SVC(C=1, kernel='linear')
{'C': 1, 'kernel': 'linear'}


In [48]:
svc_grid.best_score_

0.8313432835820895

# metrica adicional

In [53]:
#esta funcion revisa la similitud entre los valores obtenidos y los valores que se esperan
#estamos utilizando el normilize default que nos regresa la fraccion correcta de los datos prueba clasificados
#hicimos varias pruebas y este metodo nos daba de los mejores resultados en nuestras pruebas
from sklearn.metrics import accuracy_score

accuracy_score(test_y, svc.predict(test_x_vector))

0.85